# **Maestría en Inteligencia Artificial Aplicada**

## **Curso: Proyecto Integrador**

### Tecnológico de Monterrey

### Prof Dra. Grettel Barceló Alonso y Dr. Luis Eduardo Falcón Morales

## Avance V de Proyecto

## Modelo Final

## Integrantes del Equipo:
### - Erika Cardona Rojas            A01749170
### - Miriam Bönsch                  A01330346
### - Mardonio Manuel Román Ramírez  A01795265

In [ ]:
import pandas as pd

# Importando modulos
import sys
from pathlib import Path

parent_folder = str(Path.cwd().parent)
if parent_folder not in sys.path:
    sys.path.append(parent_folder)

import yaml

# Cargando Yaml
with open("../config.yaml", "r", encoding="utf-8") as file:
    config = yaml.safe_load(file)

from src import models as m

In [ ]:
#--------------------------------------------------
# Importando Datos Y Listas De Variables Relevantes
#--------------------------------------------------

# Cargando Base de Datos
df = pd.read_excel(r"../Data/DF_Pred_4.xlsx")

# Cargando las variables más importantes según RFECV 1
df_ranking = pd.read_excel('../Entregables/UdeBarcelona/RFECV.xlsx')
selected_features_xg = df_ranking.loc[df_ranking['Ranking'] == 1, 'Feature']

# Cargando las variables más importantes según RFECV 2
df_ranking = pd.read_excel('../Entregables/UdeBarcelona/RFECV_Lasso.xlsx')
selected_features_lasso = df_ranking.loc[df_ranking['Ranking'] == 1, 'Feature']

# Obteniendo variables únicas de ambos
unique_selected_features = list(set(pd.concat([selected_features_xg, selected_features_lasso])))

del df_ranking

In [ ]:
# Creando conjuntos de datos
X_full = df[unique_selected_features]
y = df.loc[:,'delta_bdnf_Int']

> ### BUSQUEDA DEL MODELO UN MEJOR MODELO
> #### SEGUN VARIABLES IMPORTANTES DE RFECV CON LASSO

In [ ]:
df_models_lasso, best_estimator_lasso = m.models_comparison_and_train_v2(X_full[selected_features_lasso], y, config['models_and_params_v2'])

In [ ]:
# Guardando resultados para analisis posterior
df_models_lasso.to_excel('../Entregables/UdeBarcelona/models_Var_Lasso.xlsx', index = False)

> ### BUSQUEDA DEL MODELO UN MEJOR MODELO
> #### SEGUN VARIABLES IMPORTANTES DE RFECV CON XGBoost

In [ ]:
df_models_XG, best_estimator_XG = m.models_comparison_and_train_v2(X_full[selected_features_xg], y, config['models_and_params_v2'])

In [ ]:
# Guardando resultados para analisis posterior
df_models_XG.to_excel('../Entregables/UdeBarcelona/models_Var_XG.xlsx', index = False)

> ### BUSQUEDA DEL MODELO UN MEJOR MODELO
> #### SEGUN VARIABLES IMPORTANTES DE RFECV CON XGBoost

In [ ]:
df_models_all, best_estimator_all = m.models_comparison_and_train_v2(X_full[unique_selected_features], y, config['models_and_params_v2'])

In [ ]:
# Guardando resultados para analisis posterior
df_models_all.to_excel('../Entregables/UdeBarcelona/models_Var_all.xlsx', index = False)

> ## Segunda Prueba de Robustez

Tras evaluar los modelos mediante Validación Cruzada, se evidencia un contraste significativo en el rendimiento dependiendo de la estrategia de selección de características utilizada:

* **Descarte de selección por RFECV (XGBoost):** Los modelos entrenados con las variables seleccionadas mediante Eliminación Recursiva de Características (RFECV) con XGBoost, así como el conjunto masivo de variables catalogadas previamente como "importantes", muestran un desempeño deficiente. Estos presentan una brecha de $R^2$ sumamente amplia entre los conjuntos de entrenamiento y validación. En nuestro escenario analítico de alta dimensionalidad y tamaño muestral reducido, esta discrepancia es un indicador inequívoco de sobreajuste severo (overfitting); la complejidad de los métodos basados en árboles ha llevado a memorizar el ruido de la muestra en lugar de extraer patrones generalizables.
* **Aprobación de la selección por Lasso:** Por otro lado, se confirma la superioridad y robustez de los modelos entrenados con el subconjunto de características filtradas mediante regularización Lasso. Gracias a la penalización L1, Lasso efectúa una selección dispersa que mitiga eficientemente la multicolinealidad, reteniendo únicamente los predictores con verdadero impacto. Esto reduce la varianza y resulta en modelos consistentes con métricas de validación altamente competitivas.

**Siguiente Fase: Verificación de Robustez mediante Y-Randomization**
Para otorgar rigor estadístico absoluto a estos resultados y descartar la posibilidad de que el alto $R^2$ sea producto de correlaciones espurias (un riesgo constante al modelar múltiples variables en pocas observaciones), construiremos un experimento de *Y-Randomization* (Permutation Test) sobre los modelos ganadores. Al desordenar aleatoriamente la variable objetivo y reevaluar iterativamente, someteremos el modelo a una doble verificación para confirmar de manera concluyente que su capacidad predictiva se basa en una señal real en los datos y no en un artefacto matemático.

De esta manera, construiremos un conjunto de 4 modelos para hacer el ensamble, a continuación el criterio de decisión:
- El modelo es REAL (Luz verde para el ensamble): El p-value es $< 0.05$. En la gráfica, veremos la campana gris (el rendimiento con los datos de polifenoles desordenados) rondando cerca del cero o en valores negativos. La línea roja (el $R^2$ de 0.9) estará muy lejos a la derecha, totalmente separada de la campana de ruido.
- El modelo es un ARTEFACTO (Descartar): El p-value es $> 0.05$. La línea roja estará peligrosamente cerca o incluso dentro de la campana gris. Esto significaría que el modelo, por su flexibilidad geométrica, es capaz de sacar un $R^2$ alto incluso cuando predice datos de salud que no tienen ninguna relación con los indicadores.


In [ ]:
# Creando Diccionario Con Los 4 Posibles Candidatos
models_dict = best_estimator_lasso
del best_estimator_lasso['KNN Regressor'], best_estimator_lasso['Decision Tree'], best_estimator_lasso['PLSRegression']

In [ ]:
m.run_y_randomization_test(models_dict, X_full[selected_features_lasso], y)